In [6]:
from geopy.geocoders import Nominatim
from geopy.geocoders import GoogleV3

import requests, json

# Anonymisation param

In [7]:

# Opening JSON file
f = open("Param/param.json")

# returns JSON object as 
# a dictionary
param = json.load(f)


In [8]:
# Initialisation

token = param['Notion_API']['token']
databaseID = param['Notion_API']['databaseID']
headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-02-22"
}

# Récupération de la base Notion des restaurants

In [309]:

readUrl = f"https://api.notion.com/v1/databases/{databaseID}/query"
res = requests.request("POST", readUrl, headers=headers)
data = res.json()
print(res.status_code)
#print(res.text)

with open('./full-properties.json', 'w', encoding='utf8') as f:
    json.dump(data, f, ensure_ascii=False)

200


# Geolocalisation

In [310]:
# Création Geocoder Google
locator = GoogleV3(api_key=param['GooglePlateform_API']['api_key'])


In [311]:
RestaurantAdresse = {'ID':[], "Name" :[], "Adresse" :[], "Lat" :[], "Lon" :[]}


for i in range(len(data['results'])):
    if data['results'][i]['properties']['Geoloc']['checkbox'] == False:
        
        RestaurantAdresse["ID"].append(data['results'][i]['id'])

        RestaurantAdresse["Name"].append(data['results'][i]['properties']['Name']['title'][0]['plain_text'])

        RestaurantAdresse["Adresse"].append(data['results'][i]['properties']['Adresse']['rich_text'][0]['text']['content'])

        location = locator.geocode(data['results'][i]['properties']['Adresse']['rich_text'][0]['text']['content'])

        if location == None:
            RestaurantAdresse["Lat"].append("tbd")

            RestaurantAdresse["Lon"].append("tbd")             

        else:
            RestaurantAdresse["Lat"].append(location.latitude)

            RestaurantAdresse["Lon"].append(location.longitude)                                


# Mettre à jour la base notion

In [312]:
# Définition de la fonctionnalité

def updateRestaurantGeoloc(pageID, headers, Lat, Lon):
    updateUrl = f"https://api.notion.com/v1/pages/{pageID}"
    updateData = {
        "properties": {
            "Geoloc": {
                "checkbox": True
                },
            "Lat": {
                "number": Lat
            },
            "Lon": {
                "number": Lon
            }
            }
        }
    data = json.dumps(updateData)
    response = requests.request("PATCH", updateUrl, headers=headers, data=data)
    print(response.status_code)

## Boucle de mise à jour geoloc

In [313]:
for i in range(len(RestaurantAdresse["ID"])):
    
    pageID = RestaurantAdresse["ID"][i]
    Lat = RestaurantAdresse["Lat"][i]
    Lon = RestaurantAdresse["Lon"][i]
    
    updateRestaurantGeoloc(pageID, headers, Lat, Lon)

200


# Projection sur carte

In [453]:
import pandas as pd
import folium
from folium import IFrame
import branca

In [454]:
readUrl = f"https://api.notion.com/v1/databases/{databaseID}/query"
res = requests.request("POST", readUrl, headers=headers)
data = res.json()
print(res.status_code)
#print(res.text)

with open('./full-properties.json', 'w', encoding='utf8') as f:
    json.dump(data, f, ensure_ascii=False)

200


In [462]:
RestaurantAdresseCarte = {'ID':[], "Name" :[], "Adresse" :[], "Lat" :[], "Lon" :[], "Atester_testé" :[], "Style" :[], "Lieu":[]}


for i in range(len(data['results'])):
    if data['results'][i]['properties']['Geoloc']['checkbox'] == True:
        
        RestaurantAdresseCarte["ID"].append(data['results'][i]['id'])

        RestaurantAdresseCarte["Name"].append(data['results'][i]['properties']['Name']['title'][0]['plain_text'])

        RestaurantAdresseCarte["Adresse"].append(data['results'][i]['properties']['Adresse']['rich_text'][0]['text']['content'])

        RestaurantAdresseCarte["Lat"].append(data['results'][i]['properties']["Lat"]['number'])
        
        RestaurantAdresseCarte["Lon"].append(data['results'][i]['properties']["Lon"]['number'])
        
        RestaurantAdresseCarte["Atester_testé"].append(data['results'][i]['properties']['A tester / testé']['status']['name'])

        RestaurantAdresseCarte["Style"].append(data['results'][i]['properties']['Style']['multi_select'])
        
        RestaurantAdresseCarte["Lieu"].append(data['results'][i]['properties']['Lieu']['select']['name'])
        

In [463]:
DfRestaurantAdresseCarte = pd.DataFrame.from_dict(RestaurantAdresseCarte)

# Création de la carte

In [615]:
map = folium.Map(location=[DfRestaurantAdresseCarte.Lat.mean(), DfRestaurantAdresseCarte.Lon.mean()], zoom_start=14, tiles=None, control_scale=True)


# Création de la fonction HTML pour la popup

In [616]:
# Notion color HEX code
NotionColorBG = {'default':'CECDCA',
                'gray':'EBECED',
                'brown':'E9E5E3',
                'orange':'FAEBDD',
                'yellow':'FBF3DB',
                'green':'DDEDEA',
                'blue':'DDEBF1',
                'purple':'EAE4F2',
                'pink':'F4DFEB',
                'red':'FBE4E4'}

In [617]:
def HtmlPopup(location_info):
    html = """ \
    <h5>"""+location_info['Name']+"""</h5><p style="line-height: 200%">"""
    

    for l in range(len(location_info['Style'])):
        html +="""<mark style="background: #"""
        html += NotionColorBG[location_info['Style'][l]['color']]
        html +=""" \
        !important"> """
        html += location_info['Style'][l]['name']
        html += """</mark>"""

    html += """</p>"""
    
    return html

# Création des layercontrol

In [618]:
GroupLieu = DfRestaurantAdresseCarte['Lieu'].drop_duplicates().sort_values().values

GroupLieuDict ={}

GroupLieuDict["All"] = FeatureGroup(name="All", overlay=False)
folium.TileLayer(tiles='OpenStreetMap', location=[DfRestaurantAdresseCarte.Lat.mean(), DfRestaurantAdresseCarte.Lon.mean()]).add_to(GroupLieuDict["All"])
GroupLieuDict["All"].add_to(map)


for i in range(len(GroupLieu)):
    
    globals()[f"GroupLieu{i}"] = FeatureGroup(name=GroupLieu[i], overlay=False)
    location = [DfRestaurantAdresseCarte[DfRestaurantAdresseCarte["Lieu"]==GroupLieu[i]].Lat.mean(), DfRestaurantAdresseCarte[DfRestaurantAdresseCarte["Lieu"]==GroupLieu[i]].Lon.mean()]
    
    folium.TileLayer(tiles='OpenStreetMap', loaction=location).add_to(globals()[f"GroupLieu{i}"])
    GroupLieuDict[GroupLieu[i]] = globals()[f"GroupLieu{i}"]
    globals()[f"GroupLieu{i}"].add_to(map)
    
    

# Plotting sur la carte

In [619]:

for index, location_info in DfRestaurantAdresseCarte.iterrows():
    if location_info["Atester_testé"] == "A tester":
        folium.Marker([location_info["Lat"], location_info["Lon"]], popup=HtmlPopup(location_info), icon=folium.Icon(icon = 'utensils', prefix='fa', color ="lightgray")).add_to(GroupLieuDict["All"])
        folium.Marker([location_info["Lat"], location_info["Lon"]], popup=HtmlPopup(location_info), icon=folium.Icon(icon = 'utensils', prefix='fa', color ="lightgray")).add_to(GroupLieuDict[location_info['Lieu']])
    
    if location_info["Atester_testé"] == "Testé":
        folium.Marker([location_info["Lat"], location_info["Lon"]], popup=HtmlPopup(location_info), icon=folium.Icon(icon = 'utensils', prefix='fa', color ="green")).add_to(GroupLieuDict["All"])
        folium.Marker([location_info["Lat"], location_info["Lon"]], popup=HtmlPopup(location_info), icon=folium.Icon(icon = 'utensils', prefix='fa', color ="green")).add_to(GroupLieuDict[location_info['Lieu']])
        
        
LayerControl().add_to(map)
        

In [622]:
map

In [223]:
map.save("home.html")